In [ ]:
'''
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install nltk
!pip install scikit-learn
!pip install sklearn
!pip install beautifulsoup4
!pip install spacy
!pip install textblob
'''

In [101]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [102]:
imdb = pd.read_csv("Data\IMDB Dataset.csv")

In [103]:
tokenizer=ToktokTokenizer()
stopwords=nltk.corpus.stopwords.words("english")
stopwords.remove("no")
stopwords.remove("not")

In [104]:
from Cleaner import denoise_text
imdb['review']=imdb['review'].apply(denoise_text)

In [105]:
from Cleaner import simple_stemmer
imdb['review']=imdb['review'].apply(simple_stemmer)

In [106]:
def remove_stopwords(text, is_lower_case=False):
    stop=set(stopwords.words('english'))
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
imdb['review']=imdb['review'].apply(remove_stopwords)

In [107]:
normalized_train_reviews=imdb.review[:40000]
normalized_test_reviews=imdb.review[40000:]

In [108]:
tv=TfidfVectorizer(min_df=0.0,max_df=1.0,use_idf=True,ngram_range=(1,3))
vector_train_reviews=tv.fit_transform(normalized_train_reviews)
vector_test_reviews=tv.transform(normalized_test_reviews)

In [109]:
lb=LabelBinarizer()
sent_data=lb.fit_transform(imdb['sentiment'])

In [110]:
train_sent=sent_data[:40000]
test_sent=sent_data[40000:]

In [111]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_tfidf=lr.fit(vector_train_reviews,train_sent)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [112]:
lr_tfidf_predict=lr.predict(vector_test_reviews)
print(lr_tfidf_predict)

[0 0 0 ... 0 0 0]


In [113]:
lr_tfidf_score=accuracy_score(test_sent,lr_tfidf_predict)
print("Our Accuracy is {}%!".format(round(lr_tfidf_score * 100)))

Our Accuracy is 89%!


In [114]:
lr_tfidf_report=classification_report(test_sent,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.89      0.89      0.89      4993
    Negative       0.89      0.89      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



## Start Bayes

In [115]:
mnb=MultinomialNB()

In [116]:
mnb_tfidf=mnb.fit(vector_train_reviews,train_sent)
print(mnb_tfidf)

MultinomialNB()


In [117]:
mnb_tfidf_predict=mnb.predict(vector_test_reviews)
print(mnb_tfidf_predict)

[0 0 0 ... 0 0 0]


In [118]:
mnb_tfidf_score=accuracy_score(test_sent,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_tfidf_score : 0.8901


In [119]:
mnb_tfidf_report=classification_report(test_sent,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.88      0.91      0.89      4993
    Negative       0.90      0.87      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



## Model Dumps

In [128]:
import pickle

In [121]:
with open ("LR_Model", "wb") as f:
    pickle.dump(lr_tfidf,f)

In [122]:
with open ("NB_Model", "wb") as f:
    pickle.dump(mnb_tfidf,f)

In [129]:
with open ("Vectorizer", "wb") as f:
    pickle.dump(tv,f)